## Data Preparation for Togo
This notebook focuses on preparing the wind and solar dataset for analysis.

The goal is to clean, structure, and validate the dataset so that it’s ready for exploration and visualization.

It follows the assignment requirements step by step: loading data, summary statistics, missing value report, outlier detection and flagging, data cleaning and imputation, and saving the cleaned dataset.

All steps use the reusable functions defined in ```data_preparation.py```, ensuring consistency and reproducibility across multiple datasets.

## Imports

In [6]:
import sys
import pandas as pd;

sys.path.append('../../../scripts')
from data_cleaning import (
    load_data,
    get_summary_report,
    calculate_zscore_and_flag_outliers,
    clean_and_impute,
    save_cleaned_data
)

## Load Raw Data

In [7]:
raw_file_path = "../../../data/togo/togo-dapaong_qc.csv"
output_dir = "../../../data/togo/"
df = load_data(raw_file_path) 

✅ 'Timestamp' column successfully converted to datetime objects.


## Observe the Data

In [8]:
print("## 📊 Head: First 5 Rows for Visual Check\n")
df.head()

## 📊 Head: First 5 Rows for Visual Check



,Timestamp,GHI,DNI,DHI,ModA,ModB,Tamb,RH,WS,WSgust,WSstdev,WD,WDstdev,BP,Cleaning,Precipitation,TModA,TModB,Comments
0,2021-10-25 00:01:00,-1.3,0.0,0.0,0.0,0.0,24.8,94.5,0.9,1.1,0.4,227.6,1.1,977,0,0.0,24.7,24.4,NaN
1,2021-10-25 00:02:00,-1.3,0.0,0.0,0.0,0.0,24.8,94.4,1.1,1.6,0.4,229.3,0.7,977,0,0.0,24.7,24.4,NaN
2,2021-10-25 00:03:00,-1.3,0.0,0.0,0.0,0.0,24.8,94.4,1.2,1.4,0.3,228.5,2.9,977,0,0.0,24.7,24.4,NaN
3,2021-10-25 00:04:00,-1.2,0.0,0.0,0.0,0.0,24.8,94.3,1.2,1.6,0.3,229.1,4.6,977,0,0.0,24.7,24.4,NaN
4,2021-10-25 00:05:00,-1.2,0.0,0.0,0.0,0.0,24.8,94.0,1.3,1.6,0.4,227.5,1.6,977,0,0.0,24.7,24.4,NaN


In [9]:
print("## ℹ️ Info: Data Types, Non-Null Counts, and Memory Usage\n")
df.info()

## ℹ️ Info: Data Types, Non-Null Counts, and Memory Usage

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525600 entries, 0 to 525599
Data columns (total 19 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   Timestamp      525600 non-null  datetime64[ns]
 1   GHI            525600 non-null  float64       
 2   DNI            525600 non-null  float64       
 3   DHI            525600 non-null  float64       
 4   ModA           525600 non-null  float64       
 5   ModB           525600 non-null  float64       
 6   Tamb           525600 non-null  float64       
 7   RH             525600 non-null  float64       
 8   WS             525600 non-null  float64       
 9   WSgust         525600 non-null  float64       
 10  WSstdev        525600 non-null  float64       
 11  WD             525600 non-null  float64       
 12  WDstdev        525600 non-null  float64       
 13  BP             525600 non-null  int64        

The raw dataset includes columns like GHI, DNI, DHI, ModA, ModB, WS, WSgust, WD, Tamb, RH, and Timestamp. Some columns may have missing values and potential outliers.

## Summary Statistics & Missing-Value Report

In [11]:
get_summary_report(df)


--- 1. Summary Statistics ---
                           Timestamp            GHI            DNI  \
count                         525600  525600.000000  525600.000000   
mean   2022-04-25 12:00:30.000000768     230.555040     151.258469   
min              2021-10-25 00:01:00     -12.700000       0.000000   
25%              2022-01-24 06:00:45      -2.200000       0.000000   
50%              2022-04-25 12:00:30       2.100000       0.000000   
75%              2022-07-25 18:00:15     442.400000     246.400000   
max              2022-10-25 00:00:00    1424.000000    1004.500000   
std                              NaN     322.532347     250.956962   

                 DHI           ModA           ModB           Tamb  \
count  525600.000000  525600.000000  525600.000000  525600.000000   
mean      116.444352     226.144375     219.568588      27.751788   
min         0.000000       0.000000       0.000000      14.900000   
25%         0.000000       0.000000       0.000000      24.200

We can see which columns have high missing values (>5%) and examine summary statistics for numeric columns. This identifies potential cleaning needs.

The Comments column contained 100% missing values, providing no useful information for analysis. Therefore, it was dropped to clean the dataset and simplify further processing.

In [12]:
df = df.drop(['Comments'], axis= 1).copy()
    
print("✅ 'Comments' column dropped due to 100% missing values.")

print("After Dropping 'Comments' Column:")
df.head()

✅ 'Comments' column dropped due to 100% missing values.
After Dropping 'Comments' Column:


,Timestamp,GHI,DNI,DHI,ModA,ModB,Tamb,RH,WS,WSgust,WSstdev,WD,WDstdev,BP,Cleaning,Precipitation,TModA,TModB
0,2021-10-25 00:01:00,-1.3,0.0,0.0,0.0,0.0,24.8,94.5,0.9,1.1,0.4,227.6,1.1,977,0,0.0,24.7,24.4
1,2021-10-25 00:02:00,-1.3,0.0,0.0,0.0,0.0,24.8,94.4,1.1,1.6,0.4,229.3,0.7,977,0,0.0,24.7,24.4
2,2021-10-25 00:03:00,-1.3,0.0,0.0,0.0,0.0,24.8,94.4,1.2,1.4,0.3,228.5,2.9,977,0,0.0,24.7,24.4
3,2021-10-25 00:04:00,-1.2,0.0,0.0,0.0,0.0,24.8,94.3,1.2,1.6,0.3,229.1,4.6,977,0,0.0,24.7,24.4
4,2021-10-25 00:05:00,-1.2,0.0,0.0,0.0,0.0,24.8,94.0,1.3,1.6,0.4,227.5,1.6,977,0,0.0,24.7,24.4


## Outlier Detection

In [13]:
df = calculate_zscore_and_flag_outliers(df)

outlier_rows = df[df['Outliers_Flag'] == True]
print("Count of rows with outlier values - ", df['Outliers_Flag'].sum())
print(outlier_rows.head(5))


Count of rows with outlier values -  9251
               Timestamp     GHI    DNI    DHI    ModA    ModB  Tamb    RH  \
4985 2021-10-28 11:06:00  1139.0  805.1  466.1  1172.0  1154.0  29.8  70.8   
5410 2021-10-28 18:11:00    -1.2    0.0    0.0     0.0     0.0  29.7  63.5   
5411 2021-10-28 18:12:00    -1.0    0.0    0.0     0.0     0.0  29.3  62.4   
5413 2021-10-28 18:14:00    -0.8    0.0    0.0     0.0     0.0  28.6  63.7   
5420 2021-10-28 18:21:00    -1.3    0.0    0.0     0.0     0.0  27.7  64.5   

       WS  WSgust  WSstdev     WD  WDstdev   BP  Cleaning  Precipitation  \
4985  2.2     2.6      0.4  298.6     13.4  977         0            0.0   
5410  6.6     9.7      1.6  122.1     14.3  976         0            0.0   
5411  6.9     8.9      1.2  128.7     10.8  976         0            0.0   
5413  7.1     8.9      1.1  127.9     14.4  976         0            0.0   
5420  6.1     8.9      1.3  123.2     14.4  977         0            0.0   

      TModA  TModB  Outliers_Fla

## Cleaning & Imputation

In [14]:
impute_cols = ['GHI', 'DNI', 'DHI', 'ModA', 'ModB', 'WS', 'WSgust']
df_cleaned = clean_and_impute(df, impute_cols)

cleaned_file_path = output_dir + "togo_cleaned.csv"
save_cleaned_data(df_cleaned, cleaned_file_path) 


✅ Cleaned Data Saved successfully to: ../../../data/togo/togo_cleaned.csv
